In [1]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")

'''
yolov8n.pt 
yolov8s.pt 
yolov8m.pt 
yolov8l.pt 
yolov8x.pt
'''


# TODO : augmentaiton, some other arguemntes
model.train(
    data="neutrophils.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    workers=-1,
    name="yolov8_neutrophils"
)

100%|██████████| 131M/131M [00:08<00:00, 16.5MB/s] 


Ultralytics 8.3.72 🚀 Python-3.12.8 torch-2.6.0+cu124 CPU (Intel Core(TM) Ultra 7 165H)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=neutrophils.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=-1, project=None, name=yolov8_neutrophils9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /home/gandalf/Documents/Code/Lung_Injury/YOLO/dataset/labels/train.cache... 205 images, 0 backgrounds, 0 corrupt: 100%|██████████| 205/205 [00:00<?, ?it/s]
val: Scanning /home/gandalf/Documents/Code/Lung_Injury/YOLO/dataset/labels/val.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8_neutrophils9/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/yolov8_neutrophils9
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.112      5.793      1.094        126        640:  15%|█▌        | 2/13 [01:40<09:14, 50.45s/it]


KeyboardInterrupt: 

In [ ]:
results = model.predict(source="tile_24555_143538.png", save=True, conf=0.1)

for result in results:
    neutrophil_count = len(result.boxes)
    print(f"Number of neutrophils detected: {neutrophil_count}")
    if neutrophil_count <= 5:
        print("Classification: Low damage")
    else:
        print("Classification: High damage")


image 1/1 /home/gandalf/Documents/Github/Mer Lab/LungHistoNet/Yolo/tile_24555_143538.png: 416x640 3 neutrophils, 49.5ms
Speed: 1.5ms preprocess, 49.5ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/yolov8_neutrophils8
Number of neutrophils detected: 3
Classification: Low damage


# CV implementation

In [ ]:
import os
from shutil import copyfile
from sklearn.model_selection import KFold
from ultralytics import YOLO

dataset_path = "/home/gandalf/Documents/Code/Lung_Injury/YOLO/dataset"
images_train_path = os.path.join(dataset_path, "images", "train")
labels_train_path = os.path.join(dataset_path, "labels", "train")

image_files = [f for f in os.listdir(images_train_path) if f.endswith(".png")]

k = 5  
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(image_files)):
    print(f"Training on Fold {fold + 1}/{k}")

    fold_path = os.path.join(dataset_path, f"fold_{fold + 1}")
    os.makedirs(fold_path, exist_ok=True)
    os.makedirs(os.path.join(fold_path, "images", "train"), exist_ok=True)
    os.makedirs(os.path.join(fold_path, "images", "val"), exist_ok=True)
    os.makedirs(os.path.join(fold_path, "labels", "train"), exist_ok=True)
    os.makedirs(os.path.join(fold_path, "labels", "val"), exist_ok=True)

    for idx in train_idx:
        image_name = image_files[idx]
        label_name = image_name.replace(".png", ".txt")
        copyfile(os.path.join(images_train_path, image_name), os.path.join(fold_path, "images", "train", image_name))
        copyfile(os.path.join(labels_train_path, label_name), os.path.join(fold_path, "labels", "train", label_name))

    for idx in val_idx:
        image_name = image_files[idx]
        label_name = image_name.replace(".png", ".txt")
        copyfile(os.path.join(images_train_path, image_name), os.path.join(fold_path, "images", "val", image_name))
        copyfile(os.path.join(labels_train_path, label_name), os.path.join(fold_path, "labels", "val", label_name))

    yaml_content = f"""
    path: {fold_path}
    train: images/train
    val: images/val

    names:
      0: neutrophil
    """
    with open(os.path.join(fold_path, "neutrophils_fold.yaml"), "w") as f:
        f.write(yaml_content)

    model = YOLO("yolov8x.pt")
    model.train(
        data=os.path.join(fold_path, "neutrophils_fold.yaml"),
        epochs=1,
        imgsz=640,
        batch=16,
        workers=-1,
        name=f"yolov8_neutrophils_fold_{fold + 1}"
    )

Training on Fold 1/5
New https://pypi.org/project/ultralytics/8.3.74 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.73 🚀 Python-3.12.9 torch-2.6.0+cu124 CPU (Intel Core(TM) Ultra 7 165H)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/home/gandalf/Documents/Code/Lung_Injury/YOLO/dataset/fold_1/neutrophils_fold.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=-1, project=None, name=yolov8_neutrophils_fold_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug

train: Scanning /home/gandalf/Documents/Code/Lung_Injury/YOLO/dataset/fold_1/labels/train.cache... 79 images, 0 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<?, ?it/s]
val: Scanning /home/gandalf/Documents/Code/Lung_Injury/YOLO/dataset/fold_1/labels/val.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]

Plotting labels to runs/detect/yolov8_neutrophils_fold_1/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/yolov8_neutrophils_fold_1
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.808      5.921     0.9832        120        640:  20%|██        | 1/5 [00:57<03:51, 57.87s/it]